In [127]:
import pandas as pd
from pathlib import Path

In [159]:
def get_single_year_abs_data(filepath):

    year = Path(filepath).stem[0:4] 
    year_str = str(year)

    df_raw = pd.read_csv(
        filepath,
        header=6,
        names=["Postcode", "Fuel Type", "Vehicle Type", year_str],
        #dtype = {'Postcode': str, 'Fuel Type': str, 'Vehicle Type': str, year_str: int},
        #dtype=object,
        index_col=False,
        skipfooter=13,
        engine="python",
        #nrows=1000,
    )
    df = df_raw.copy()
    
    # ensure string type on this column
    df = df.astype({'Postcode':'string'})
    df = df.astype({year_str:'int64'})

    # fill nans 
    df.fillna(method="ffill", axis=0, inplace=True)

    # drop rows from df where Postcode is #TOT or Total
    df.drop(df[df["Postcode"] == "#TOT"].index, inplace=True)
    df.drop(df[df["Postcode"] == "Total"].index, inplace=True)

    # add consistent leading zeros to postcodes
    df['Postcode'] = df['Postcode'].str.zfill(4)

    vehicle_type_mapping = {
        "Passenger Vehicles": "PV",
        "Light Commercial Vehicles": "LCV",
    }
    for v in vehicle_type_mapping:
        df["Vehicle Type"] = df["Vehicle Type"].str.replace(v, vehicle_type_mapping[v])

    # pivot to make merging columns eaiser
    df = df.pivot(index=["Postcode","Vehicle Type"], columns=["Fuel Type"], values=year_str)

    # combine petrol leaded and unleaded
    df['Petrol']=df['Petrol leaded']+df['Petrol unleaded']
    df.drop(['Petrol leaded','Petrol unleaded'],axis=1,inplace=True)

    # combine other and unknown
    unknown_col = 'Unknown'
    if int(year) < 2018:
        unknown_col = 'Other and unknown'

    df['Other']=df['Other']+df[unknown_col]
    df.drop([unknown_col],axis=1,inplace=True)

    df.rename({"LPG/Other gases": "LPG"}, axis=1, inplace=True)

    # unpivot now that we are done
    df.reset_index(inplace=True)
    df = df.melt(id_vars=["Postcode","Vehicle Type"], var_name="Fuel Type", value_name=year_str)

    return df

In [160]:
df_2021 = get_single_year_abs_data("ABS/2021-abs-fuel-type-no-state.csv")
df_2020 = get_single_year_abs_data("ABS/2020-abs-fuel-type-no-state.csv")
df_2013 = get_single_year_abs_data("ABS/2013-aps-fuel-type-no-state.csv")

In [161]:
def get_abs_data():
    files = sorted(Path("ABS").glob("*.csv"))
    return_df = pd.DataFrame()
    for f in files:
        df = get_single_year_abs_data(f)
        if not return_df.empty:
            return_df = return_df.merge(df, how="outer", on=["Postcode", "Vehicle Type","Fuel Type"])
        else:
            return_df = df
    
    return_df.to_csv("abs_df.csv")
    return return_df

abs_df = get_abs_data()

In [170]:
abs_long = abs_df.melt(id_vars=["Postcode","Vehicle Type","Fuel Type"], var_name="Year", value_name="Count")
fuel_type_df = abs_long.groupby(["Fuel Type", "Year"]).sum()
fuel_type_df.reset_index(inplace=True)

In [169]:
import plotly.express as px
fig1 = px.area(fuel_type_df, x="Year", y='Count', color='Fuel Type')
fig1.show()


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=75de7644-8bd4-4ecc-bdb2-2c9ef0ed94e0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>